In [ ]:
import xarray as xr
from obspy import UTCDateTime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#code to pull historic buoy data using the ERDDAP url (NOAA CoastWatch)
#need to know the station number and the start and end time
#whatever columns the buoy does not have, it will put NaN instead
#what each of the column variables means: https://www.ndbc.noaa.gov/faq/measdes.shtml
#forks washington
#station = "46041"
#start = "2025-12-05T00:00:00Z"
#end   = "2025-12-11T00:00:00Z"
#santa barbara
station = "46053"
start = "2025-12-21T00:00:00Z"
end   = "2025-12-27T00:00:00Z"

url = (
    "https://coastwatch.pfeg.noaa.gov/erddap/tabledap/"
    "cwwcNDBCMet.csv?"
    "station,longitude,latitude,time,"
    "wd,wspd,gst,wvht,dpd,apd,mwd,bar,"
    "atmp,wtmp,dewp,vis,ptdy,tide,wspu,wspv"
    f"&station=%22{station}%22"
    f"&time%3E={start}&time%3C={end}"
)

#to download the csv and look at it in excel, print the url below
#print(url)

# skip first row (units) and parse time
data = pd.read_csv(url, skiprows=[1], parse_dates=["time"])

In [ ]:
data['wvht'] = pd.to_numeric(data['wvht'], errors='coerce')
data['wspd'] = pd.to_numeric(data['wspd'], errors='coerce')
data = data.set_index('time')

#go from samples every few minutes to ever yhour, take average
hourly = data[['wvht', 'wspd']].resample('1H').mean()

#turn pandas lists into numpy array
buoytime_hourly = hourly.index.to_numpy(dtype='datetime64[h]')
sigwaveheight_hourly = hourly['wvht'].to_numpy()
offshorewind_hourly = hourly['wspd'].to_numpy()

In [ ]:
# assign variables to each column (dont need this code)
# https://www.ndbc.noaa.gov/faq/measdes.shtml
# https://erddap.aoml.noaa.gov/hdb/erddap/info/NDBC_BUOY_1997_present/index.html#:~:text=The%20zonal%20wind%20speed%20(m/s)%20indicates%20the,derived%20from%20Wind%20Direction%20and%20Wind%20Speed.

time = data["time"]
wind_dir = data["wd"]            # degrees_true
wind_speed = data["wspd"]        # m/s
wind_gust = data["gst"]          # m/s
wave_height = data["wvht"]       # m    (significant wave height)
wave_period = data["dpd"]        # s    (dominant wave period)
av_wave_period = data["apd"]     # s    (average wave period)
wave_dir = data["mwd"]           # degrees_true
pressure = data["bar"]           # hPa
air_temp = data["atmp"]          # °C
water_temp = data["wtmp"]        # °C
zonal_wind = data["wspu"]        # m/s  (indicates the u component of where the wind is going)
merid_wind = data["wspv"]        # m/s  (indicates the v component of where the wind is going)


In [ ]:
#check if there are NaN values in a column
#pd.Series(wave_height[0:20]).isna()

In [ ]:
#plot wave heights as a function of time
plt.figure(figsize=(12,5))
plt.plot(buoytime_hourly, sigwaveheight_hourly, marker='.', linestyle='solid', color='black')
plt.xlabel("Time")
plt.ylabel("Wave Height (m)")
plt.title(f"Buoy {station} - Wave Height - {start} to {end}")
plt.show()

In [ ]:
# remove NaNs with mask from both arrays and plot that way
mask = ~np.isnan(wave_height)
plt.figure(figsize=(12,5))
plt.plot(np.array(time)[mask], np.array(wave_height)[mask],
         linestyle='solid', linewidth=1.5, color='black')
plt.xlabel("Time")
plt.ylabel("Wave Height (m)")
plt.title(f"Buoy {station} - Wave Height")
plt.show()

In [ ]:
#plot wind speeds as a function of time
plt.figure(figsize=(12,5))
plt.plot(time, wind_speed,linestyle='solid', linewidth=1.5, color='black')
plt.xlabel("Time")
plt.ylabel("Wind Speeds (m/s)")
plt.title(f"Buoy {station} - Wind Speed")
plt.show()

In [ ]:
#code to upload tide and inland wind speeds from csv files

#tides found here https://tidesandcurrents.noaa.gov/waterlevels.html?id=9442396&units=metric&bdate=20251201&edate=20251231&timezone=GMT&datum=MLLW&interval=6&action=data
#wind speed downloaded here https://tidesandcurrents.noaa.gov/met.html?bdate=20251201&edate=20251231&units=metric&timezone=GMT&id=9442396&interval=h&action=data
#downloaded tide values against MLLW
#downlaoded wind speeds every hour
#La Push, Quillayute River, WA - Station ID: 9442396 
#Santa Barbara, CA - Station ID: 9411340

#read in the csv files
#tide_data = pd.read_csv('CO-OPS_9442396_wl.csv')
#wind_inland = pd.read_csv('CO-OPS_9442396_met6.csv')

tide_data = pd.read_csv('CO-OPS_9411340_wl.csv')
wind_inland = pd.read_csv('CO-OPS_9411340_met.csv')

#start with tide file
#turn time into datetime format
time_td = pd.to_datetime(tide_data['Date'] + 'T' + tide_data['Time (GMT)'])
#put water level and time into one array
#if ‘coerce’, then invalid parsing will be set as NaN
tide_data['DateTime'] = time_td
tide_data['Verified (m)'] = pd.to_numeric(tide_data['Verified (m)'], errors='coerce')
tide_data = tide_data.set_index('DateTime')

#go from every 6 minutes to ever hour, take average
tide_hourly = tide_data['Verified (m)'].resample('1H').mean()

#turn pandas list into numpy array
time_hourly = tide_hourly.index.to_numpy(dtype='datetime64[h]')
water_level_hourly = tide_hourly.to_numpy()

plt.figure(figsize=(12, 4))
plt.plot(time_hourly, water_level_hourly)
plt.xlabel('Time')
plt.ylabel('Water Level (m)')
plt.title('Hourly Averaged Tide Elevation')
plt.grid(True)
plt.show()


In [ ]:
#to subset the data by certain days, using pandas
tide_subset = tide_hourly['2025-12-05':'2025-12-11']

#doing the same subset but with numpy
start = np.datetime64('2025-12-21')
end   = np.datetime64('2025-12-27')

mask = (time_hourly >= start) & (time_hourly <= end)

time_subset = time_hourly[mask]
water_subset = water_level_hourly[mask]

#plot subset
plt.figure(figsize=(10, 4))
plt.plot(time_subset, water_subset)
#plt.plot(tide_subset.index, tide_subset.values)
plt.xlabel('Time')
plt.ylabel('Water Level (m)')
plt.title('Hourly Averaged Tide Elevation (matches Raspberry Shake)')
plt.grid(True)
plt.show()

#water_level = tide_data['Verified (m)']

#plt.plot(time_td, water_level)
#plt.xlabel("Time")
#plt.ylabel("Tide Water Level (m)")
#plt.title("Tide Gauge")
#plt.show()

In [ ]:
time_wd = wind_inland['Date'] + 'T' + wind_inland['Time (GMT)']
time_wd = pd.to_datetime(time_wd)

wind_inland['DateTime'] = time_wd
wind_inland['Wind Speed (m/s)'] = pd.to_numeric(wind_inland['Wind Speed (m/s)'], errors='coerce')
wind_inland = wind_inland.set_index('DateTime')

#go from every 6 minutes to ever hour, take average
windin_hourly = wind_inland['Wind Speed (m/s)'].resample('1H').mean()

#turn pandas list into numpy array
timew_hourly = windin_hourly.index.to_numpy(dtype='datetime64[h]')
inland_wind_hourly = windin_hourly.to_numpy()

plt.figure(figsize=(12, 4))
plt.plot(timew_hourly, inland_wind_hourly)
plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.title('Hourly Averaged Wind Speed from Tide Station')
plt.grid(True)
plt.show()

maskwind = (timew_hourly >= start) & (timew_hourly <= end)

timewind_subset = timew_hourly[maskwind]
wind_subset = inland_wind_hourly[maskwind]

#plot subset
plt.figure(figsize=(10, 4))
plt.plot(timewind_subset, wind_subset)
#plt.plot(tide_subset.index, tide_subset.values)
plt.xlabel('Time')
plt.ylabel('Water Level (m)')
plt.title('Hourly Inland Wind Speed (matches Raspberry Shake)')
plt.grid(True)
plt.show()


In [ ]:
#plot all the datasets together

fig, ax = plt.subplots(figsize=(12, 4)) 
plt.plot(buoytime_hourly, sigwaveheight_hourly, color='r', label='significant wave height (m)')
plt.plot(buoytime_hourly, offshorewind_hourly, color='g', label='offshore wind speed (m/s)')
plt.plot(time_subset, water_subset, color='b', label='tide elevation (m OD)')
plt.plot(timewind_subset, wind_subset, color='c', label='inland wind speed (m/s)')
plt.xlabel('Time')
plt.ylabel('Wave height / Tide elevation / Wind velocity')
plt.title('Fork WA Hourly Averaged Buoy + Tide Gauge Data')
plt.grid(True)
#plt.legend(loc='upper left')
ax.legend(loc='upper left', bbox_to_anchor=(1, 0.7), ncol=1, fancybox=True, shadow=True)
#plt.savefig('buoy_tide1.png')
plt.show()